[Link](https://www.kaggle.com/competitions/digit-recognizer/data) to Competition

In [44]:
#|default_exp app

In [45]:
#| export
import os
from pathlib import Path
import pandas as pd
import numpy as np
from fastai.vision.all import *

matplotlib.rc('image', cmap='Greys')

In [46]:
#| export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
creds = ''

In [47]:
#| export
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [48]:
#| export
path = Path('digit-recognizer')

In [49]:
#| export
if not iskaggle and not path.exists():
    import zipfile, kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [50]:
#| export
if iskaggle:
    path = Path('../input/digit-recognizer')
    ! pip install -q dataset

For each of the 28000 images in the test set, output a single line containing the ImageId and the digit you predict. For example, if you predict that the first image is of a 3, the second image is of a 7, and the third image is of a 8, then your submission file would look like:

`ImageId,Label`<br>
`1,3`<br>
`2,7`<br>
`3,8` <br>
`(27997 more lines)`<br>

In [51]:
train_full = pd.read_csv(path/'train.csv')

In [52]:
train_full.head()


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
label = train_full['label']

In [54]:
image_df = train_full.iloc[:, 1:]

In [55]:
image_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
image_df.shape

(42000, 784)

In [57]:
# rows = (len(image_df.axes[1]))
# cols = (len(image_df.axes[1]))
# rows, cols

In [58]:
np_image_array = image_df.values
np_image_array, np_image_array.shape

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 (42000, 784))

In [59]:
image_tens = [tensor(x) for x in np_image_array]

In [60]:
train_x = torch.stack(image_tens)
train_x.shape

torch.Size([42000, 784])

In [61]:
train_y = tensor(label).unsqueeze(1)
train_y, train_y.shape

(tensor([[1],
         [0],
         [1],
         ...,
         [7],
         [6],
         [9]]),
 torch.Size([42000, 1]))

In [62]:
dset = list(zip(train_x, train_y))
x,y = dset[0]
x.shape, y

(torch.Size([784]), tensor([1]))

In [70]:
df_image = pd.DataFrame(train_x[100].reshape(28, 28))
df_image.style.set_properties(**{'font-size':'6pt'}).background_gradient('Greys')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,62,176,254,254,228,38,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,4,87,204,245,253,171,140,254,192,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,21,128,220,253,230,164,114,24,37,254,245,186,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,14,135,230,253,238,112,20,0,0,141,253,254,253,229,20,0,0,0,0,0
